We were able to get a dataframe with game stats for 5 seasons of mens college basketball.
We want to train a model using regular season data and use that to predict March Madness.
We will have to calculate teams season avgs to input into March Madness games.

In [1]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('NCAABasketballData.csv', low_memory = False)

In [ ]:
df.h_market

In [ ]:
df.a_market

In [ ]:
df.tournament.value_counts()

In [ ]:
regular_df = df[df.tournament != 'NCAA']
regular_df.shape

In [ ]:
mm_df = df[df.tournament == 'NCAA']
mm_df.shape

In [ ]:
list(df.columns)

In [ ]:
df.tournament_round.isna().sum()

In [ ]:
len(df.h_id.unique())

In [ ]:
df.scheduled_date

In [ ]:
#from sportsipy.ncaab.teams import Teams
schools = []
#teams = Teams()
for team in teams:
    schools.append(team.name)  
schools

In [ ]:
len(schools)

In [ ]:
# Make replace all spaces with dashes so it matches the url of the NCAA_data file
for i in range(len(schools)):
    schools[i] = "-".join(schools[i].split())
# Now make it all lower case
schools = (map(lambda x: x.lower(), schools))
schools = list(schools)
# Now remove all parentheses and '&' charachters
schools = [schools[i].replace('(', '').replace(')', '').replace('&', '') for i in range(len(schools))]
# remove all '&'s
schools

In [ ]:
# While running the code below an error if the name of the school is not the same as the way it is in the url.
# We can fix each problem as they come up.
# Some schools are not in the NCAAb_data file altogether and we'll have to remove those from our list.

In [ ]:
schools[13] = 'arkansas-little-rock'
schools[42] = 'california-davis'
schools[43] = 'california-irvine'
schools[44] = 'california-riverside'
schools[45] = 'california-san-diego'
schools[46] = 'california-santa-barbara'
schools[47] = 'california'
schools[59] = 'citadel'

In [ ]:
df = pd.read_csv('NCAAB_data/abilene-christian.csv')
for i in schools[1:]:
    df = df.append(pd.read_csv(f'NCAAB_data/{i}.csv'))

In [ ]:
df.shape

In [ ]:
# get list of all the file names in our NCAAB_data folder
from os import listdir
files = listdir('NCAAB_data')

In [ ]:
len(files)

In [ ]:
for f in range(len(files)): 
    if files[f].endswith('pki'):
        files = files.remove(files[f])
files

In [101]:
off_stats_17 = pd.read_csv('NCAAB_data/2017-18_Advanced_Stats.csv')
off_stats_17.columns = off_stats_17.iloc[0]
off_stats_17.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 352 entries, 0 to 351
Data columns (total 34 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Rk      352 non-null    object 
 1   School  352 non-null    object 
 2   G       352 non-null    object 
 3   W       352 non-null    object 
 4   L       352 non-null    object 
 5   W-L%    352 non-null    object 
 6   SRS     352 non-null    object 
 7   SOS     352 non-null    object 
 8   nan     0 non-null      float64
 9   W       352 non-null    object 
 10  L       352 non-null    object 
 11  nan     0 non-null      float64
 12  W       352 non-null    object 
 13  L       352 non-null    object 
 14  nan     0 non-null      float64
 15  W       352 non-null    object 
 16  L       352 non-null    object 
 17  nan     0 non-null      float64
 18  Tm.     352 non-null    object 
 19  Opp.    352 non-null    object 
 20  nan     0 non-null      float64
 21  Pace    352 non-null    object 
 22  OR

In [102]:
indices = [1,21,22,23,24,25,26,27,28,29,30,31,32,33]

In [103]:
off_stats_17 = off_stats_17.iloc[:, indices]
off_stats_17

,School,Pace,ORtg,FTr,3PAr,TS%,TRB%,AST%,STL%,BLK%,eFG%,TOV%,ORB%,FT/FGA
0,School,Pace,ORtg,FTr,3PAr,TS%,TRB%,AST%,STL%,BLK%,eFG%,TOV%,ORB%,FT/FGA
1,Abilene Christian,71.5,102.3,.309,.350,.549,49.7,55.4,11.3,11.6,.521,17.7,28.0,.217
2,Air Force,67.7,100.8,.318,.431,.527,48.7,60.7,9.5,8.1,.490,16.3,27.5,.233
3,Akron,69.0,102.7,.319,.467,.547,49.2,52.7,8.5,7.5,.518,17.3,27.5,.222
4,Alabama A&M,68.2,88.2,.314,.354,.480,48.2,50.5,5.8,3.9,.450,20.9,29.9,.203
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
347,Wright State NCAA,69.9,101.4,.358,.348,.529,51.8,52.7,9.4,6.5,.491,16.3,28.9,.256
348,Wyoming,74.7,103.3,.403,.450,.564,47.1,59.3,8.4,9.8,.523,16.1,19.2,.298
349,Xavier NCAA,72.3,115.5,.410,.358,.601,55.1,57.3,8.0,8.4,.559,15.2,30.2,.320
350,Yale,70.1,105.5,.307,.422,.562,49.8,61.0,9.3,8.2,.533,16.5,26.0,.221


In [104]:
def_stats_17 = pd.read_csv('NCAAB_data/2017-18_Opponent_data.csv')
def_stats_17.columns = def_stats_17.iloc[0]
def_stats_17

,Rk,School,G,W,L,W-L%,SRS,SOS,NaN,W,...,3PAr,TS%,TRB%,AST%,STL%,BLK%,eFG%,TOV%,ORB%,FT/FGA
0,Rk,School,G,W,L,W-L%,SRS,SOS,NaN,W,...,3PAr,TS%,TRB%,AST%,STL%,BLK%,eFG%,TOV%,ORB%,FT/FGA
1,1,Abilene Christian,32,16,16,.500,-9.14,-6.82,NaN,8,...,.374,.540,50.3,48.1,8.9,7.5,.499,19.5,28.5,.298
2,2,Air Force,31,12,19,.387,-4.31,1.72,NaN,6,...,.431,.576,51.3,57.2,8.3,10.6,.549,18.3,27.8,.234
3,3,Akron,32,14,18,.438,-6.82,-1.92,NaN,6,...,.401,.567,50.8,51.1,9.0,11.7,.532,16.2,28.0,.277
4,4,Alabama A&M,31,3,28,.097,-23.97,-8.04,NaN,3,...,.323,.562,51.8,52.2,11.2,13.8,.540,14.4,32.4,.204
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
347,347,Wright State NCAA,35,25,10,.714,0.58,-3.64,NaN,14,...,.394,.516,48.2,51.5,8.3,7.8,.483,19.2,25.0,.207
348,348,Wyoming,33,20,13,.606,4.26,3.54,NaN,10,...,.346,.546,52.9,51.3,8.0,7.3,.508,16.9,25.6,.268
349,349,Xavier NCAA,35,29,6,.829,19.06,9.43,NaN,15,...,.411,.532,44.9,52.1,9.3,9.6,.499,14.0,22.9,.195
350,350,Yale,31,16,15,.516,-4.19,-2.83,NaN,9,...,.352,.552,50.2,51.9,9.8,7.7,.519,16.1,26.3,.238


In [105]:
def_stats_17.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 352 entries, 0 to 351
Data columns (total 34 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Rk      352 non-null    object 
 1   School  352 non-null    object 
 2   G       352 non-null    object 
 3   W       352 non-null    object 
 4   L       352 non-null    object 
 5   W-L%    352 non-null    object 
 6   SRS     352 non-null    object 
 7   SOS     352 non-null    object 
 8   nan     0 non-null      float64
 9   W       352 non-null    object 
 10  L       352 non-null    object 
 11  nan     0 non-null      float64
 12  W       352 non-null    object 
 13  L       352 non-null    object 
 14  nan     0 non-null      float64
 15  W       352 non-null    object 
 16  L       352 non-null    object 
 17  nan     0 non-null      float64
 18  Tm.     352 non-null    object 
 19  Opp.    352 non-null    object 
 20  nan     0 non-null      float64
 21  Pace    352 non-null    object 
 22  OR

In [106]:
def_stats_17 = def_stats_17.iloc[:, indices[1:]]
def_stats_17

,Pace,ORtg,FTr,3PAr,TS%,TRB%,AST%,STL%,BLK%,eFG%,TOV%,ORB%,FT/FGA
0,Pace,ORtg,FTr,3PAr,TS%,TRB%,AST%,STL%,BLK%,eFG%,TOV%,ORB%,FT/FGA
1,71.5,98.8,.422,.374,.540,50.3,48.1,8.9,7.5,.499,19.5,28.5,.298
2,67.7,106.5,.328,.431,.576,51.3,57.2,8.3,10.6,.549,18.3,27.8,.234
3,69.0,107.9,.383,.401,.567,50.8,51.1,9.0,11.7,.532,16.2,28.0,.277
4,68.2,111.5,.298,.323,.562,51.8,52.2,11.2,13.8,.540,14.4,32.4,.204
...,...,...,...,...,...,...,...,...,...,...,...,...,...
347,69.9,93.6,.292,.394,.516,48.2,51.5,8.3,7.8,.483,19.2,25.0,.207
348,74.7,101.8,.372,.346,.546,52.9,51.3,8.0,7.3,.508,16.9,25.6,.268
349,72.3,102.3,.256,.411,.532,44.9,52.1,9.3,9.6,.499,14.0,22.9,.195
350,70.1,104.4,.328,.352,.552,50.2,51.9,9.8,7.7,.519,16.1,26.3,.238


In [107]:
def_columns = list(def_stats_17.columns)
for i in range(len(def_columns)):
    def_columns[i] = 'opp'+def_columns[i]

In [108]:
def_stats_17.columns = def_columns
def_stats_17

,oppPace,oppORtg,oppFTr,opp3PAr,oppTS%,oppTRB%,oppAST%,oppSTL%,oppBLK%,oppeFG%,oppTOV%,oppORB%,oppFT/FGA
0,Pace,ORtg,FTr,3PAr,TS%,TRB%,AST%,STL%,BLK%,eFG%,TOV%,ORB%,FT/FGA
1,71.5,98.8,.422,.374,.540,50.3,48.1,8.9,7.5,.499,19.5,28.5,.298
2,67.7,106.5,.328,.431,.576,51.3,57.2,8.3,10.6,.549,18.3,27.8,.234
3,69.0,107.9,.383,.401,.567,50.8,51.1,9.0,11.7,.532,16.2,28.0,.277
4,68.2,111.5,.298,.323,.562,51.8,52.2,11.2,13.8,.540,14.4,32.4,.204
...,...,...,...,...,...,...,...,...,...,...,...,...,...
347,69.9,93.6,.292,.394,.516,48.2,51.5,8.3,7.8,.483,19.2,25.0,.207
348,74.7,101.8,.372,.346,.546,52.9,51.3,8.0,7.3,.508,16.9,25.6,.268
349,72.3,102.3,.256,.411,.532,44.9,52.1,9.3,9.6,.499,14.0,22.9,.195
350,70.1,104.4,.328,.352,.552,50.2,51.9,9.8,7.7,.519,16.1,26.3,.238


In [111]:
team_stats_17 = pd.concat([off_stats_17, def_stats_17], axis=1)
team_stats_17.drop([0], inplace=True)

In [113]:
team_stats_17

,School,Pace,ORtg,FTr,3PAr,TS%,TRB%,AST%,STL%,BLK%,...,opp3PAr,oppTS%,oppTRB%,oppAST%,oppSTL%,oppBLK%,oppeFG%,oppTOV%,oppORB%,oppFT/FGA
1,Abilene Christian,71.5,102.3,.309,.350,.549,49.7,55.4,11.3,11.6,...,.374,.540,50.3,48.1,8.9,7.5,.499,19.5,28.5,.298
2,Air Force,67.7,100.8,.318,.431,.527,48.7,60.7,9.5,8.1,...,.431,.576,51.3,57.2,8.3,10.6,.549,18.3,27.8,.234
3,Akron,69.0,102.7,.319,.467,.547,49.2,52.7,8.5,7.5,...,.401,.567,50.8,51.1,9.0,11.7,.532,16.2,28.0,.277
4,Alabama A&M,68.2,88.2,.314,.354,.480,48.2,50.5,5.8,3.9,...,.323,.562,51.8,52.2,11.2,13.8,.540,14.4,32.4,.204
5,Alabama-Birmingham,69.4,109.8,.291,.334,.575,54.8,59.3,7.7,11.6,...,.437,.524,45.2,53.2,10.0,7.7,.498,15.9,24.6,.183
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
347,Wright State NCAA,69.9,101.4,.358,.348,.529,51.8,52.7,9.4,6.5,...,.394,.516,48.2,51.5,8.3,7.8,.483,19.2,25.0,.207
348,Wyoming,74.7,103.3,.403,.450,.564,47.1,59.3,8.4,9.8,...,.346,.546,52.9,51.3,8.0,7.3,.508,16.9,25.6,.268
349,Xavier NCAA,72.3,115.5,.410,.358,.601,55.1,57.3,8.0,8.4,...,.411,.532,44.9,52.1,9.3,9.6,.499,14.0,22.9,.195
350,Yale,70.1,105.5,.307,.422,.562,49.8,61.0,9.3,8.2,...,.352,.552,50.2,51.9,9.8,7.7,.519,16.1,26.3,.238


In [112]:
team_stats_17.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 351 entries, 1 to 351
Data columns (total 27 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   School     351 non-null    object
 1   Pace       351 non-null    object
 2   ORtg       351 non-null    object
 3   FTr        351 non-null    object
 4   3PAr       351 non-null    object
 5   TS%        351 non-null    object
 6   TRB%       351 non-null    object
 7   AST%       351 non-null    object
 8   STL%       351 non-null    object
 9   BLK%       351 non-null    object
 10  eFG%       351 non-null    object
 11  TOV%       351 non-null    object
 12  ORB%       351 non-null    object
 13  FT/FGA     351 non-null    object
 14  oppPace    351 non-null    object
 15  oppORtg    351 non-null    object
 16  oppFTr     351 non-null    object
 17  opp3PAr    351 non-null    object
 18  oppTS%     351 non-null    object
 19  oppTRB%    351 non-null    object
 20  oppAST%    351 non-null    objec